This notebook builds the ensemble of our 2 best methods, to further improve predictions, by averaging the probability attributed by each one of the methods to the test set images. This improved our best results letting us achieve our best score of 0.97111 on the test set.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

We import libraries, set the seed and the working directory.

In [ ]:
import os
import tensorflow as tf
import numpy as np

SEED = 27091997
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

We load the models that we want to use for the ensemble: 

*   A model obtained with EfficientnetB5
*   The models of the two steps of the double classifier using EfficientnetB5 



In [ ]:
model3 = tf.keras.models.load_model('/content/drive/My Drive/modelloEfficientNetB5') # EfficientnetB5
model4 = tf.keras.models.load_model('/content/drive/My Drive/ModelloefficientnetB5double1') # First step of the double classifier
model5 = tf.keras.models.load_model('/content/drive/My Drive/ModelloefficientnetB5double2') # Second step of the double classifier

We set the directory and we unzip the file containing our data in order to import the images in the test set.

In [ ]:
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

Output streaming troncato alle ultime 5000 righe.
  inflating: __MACOSX/MaskDataset/training/._17387.jpg  
  inflating: MaskDataset/training/17393.jpg  
  inflating: __MACOSX/MaskDataset/training/._17393.jpg  
  inflating: MaskDataset/training/15584.jpg  
  inflating: __MACOSX/MaskDataset/training/._15584.jpg  
  inflating: MaskDataset/training/16933.jpg  
  inflating: __MACOSX/MaskDataset/training/._16933.jpg  
  inflating: MaskDataset/training/11084.jpg  
  inflating: __MACOSX/MaskDataset/training/._11084.jpg  
  inflating: MaskDataset/training/13863.jpg  
  inflating: __MACOSX/MaskDataset/training/._13863.jpg  
  inflating: MaskDataset/training/11912.jpg  
  inflating: __MACOSX/MaskDataset/training/._11912.jpg  
  inflating: MaskDataset/training/13877.jpg  
  inflating: __MACOSX/MaskDataset/training/._13877.jpg  
  inflating: MaskDataset/training/15989.jpg  
  inflating: __MACOSX/MaskDataset/training/._15989.jpg  
  inflating: MaskDataset/training/15751.jpg  
  inflating: __MACOSX/M

In [ ]:
dataset_dir = os.path.join(cwd,'MaskDataset')

In [ ]:
#we fix the image shape as in the models we use
# img shape
img_h = 456
img_w = 456


Now we compute predictions: first we compute the probabilities to belong to each of the 3 classes for both models, then we use their mean to choose the final class of each test image. We also tried to compute predictions with the most voted class for each image introducing a third model in the ensemble, or weighting by meaningful proportions, but we think this way is more clear, and it also achieves slightly better results.




In [ ]:
# We compute the probabilities to belong to each of the 3 classes for the images in the test set according to the first model
from PIL import Image
image_filenames = next(os.walk(os.path.join(dataset_dir, 'test')))[2]

results3 = {}
for image_name in image_filenames:

   img = Image.open(os.path.join(dataset_dir, 'test')+'/'+image_name).convert('RGB')
   img=img.resize((img_h, img_w))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   img_array = np.true_divide(img_array,255)
   predictions=model3.predict(img_array)
   results3[image_name] = predictions

In [ ]:
#We compute the probabilities to belong to each of the 3 classes for the images in the test set according to the double classifier
image_filenames = next(os.walk(os.path.join(dataset_dir, 'test')))[2]

results4 = {}
for image_name in image_filenames:

   img = Image.open(os.path.join(dataset_dir, 'test')+'/'+image_name).convert('RGB')
   img=img.resize((img_h, img_w))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   img_array = np.true_divide(img_array,255)
   predictionsloc1=model4.predict(img_array)
   prediction = np.argmax(predictionsloc1)   
   if prediction!=0:                             
     predictionsloc2=model5.predict(img_array)
     prediction = np.argmax(predictionsloc2)
     if prediction==1:
       prediction=2
     else:
       prediction=1
     predictions=[predictionsloc1[0][0],predictionsloc1[0][1]*predictionsloc2[0][0],predictionsloc1[0][1]*predictionsloc2[0][1]]  
   else:
     predictions=[predictionsloc1[0][0],predictionsloc1[0][1]/2,predictionsloc1[0][1]/2]  
   results4[image_name] = predictions
# For all images the probability to belong to class 0 is in the output of the first classifier.
# For the images classified as 0 in the first classifier we assume that the probability to belong to the final classes 1 and 2 is the half of
# the probability to belong to the class 1 in the first.
# For images classified as 1 in the first classifier we compute the probability to belong to the final classes 1 and 2 as the product of 
# the probability to belong to class 1 in the first classifier and the probability to belong to class 0, or 1, in the second, respectively.

In [ ]:
#We compute the final predictions: the new probabilities to belong to each of the 3 classes are the mean of the probabilities of the 2
#different models that we use. Then we use the argmax to find the most probable class.
partial1={}
results={}
for k in set(results1):
  partial1[k]=[results3[k][0][0]+ results4[k][0],results3[k][0][1]+ results4[k][1],results3[k][0][2]+ results4[k][2]]
  results[k] = np.argmax(partial1[k]) 

In the end we create the csv file containing the results

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(results, '/content/drive/My Drive')